# PyPRT - ArcPy Case Study

Description: this notebook is used to test the arcpy module in relation to pyprt module.

In [1]:
import sys, os
SDK_PATH = os.path.join(os.getcwd(), "..", "install", "bin")
sys.path.append(SDK_PATH)

print(SDK_PATH)

In [2]:
import numpy as np

In [3]:
import pyprt

In [3]:
import arcpy

In [4]:
VAL = pyprt.print_val(4074)
print("\nTest Function: it should print 407.")
print(VAL)


Test Function: it should print 407.
4074


In [4]:
CS_FOLDER = os.getcwd()
def asset_file(filename):
    return os.path.join(CS_FOLDER, filename)

In [5]:
print("\nInitializing PRT.")
pyprt.initialize_prt(SDK_PATH)

if(not pyprt.is_prt_initialized()):
    raise Exception("PRT is not initialized")


Initializing PRT.


In [5]:
mod = pyprt.ModelGenerator(asset_file("simple_scene_0.obj"), asset_file("simple_rule2019.rpk"), ["ruleFile:string=bin/simple_rule2019.cgb", "startRule:string=Default$Footprint"], ["baseName:string=theModelSuper"])

if(mod.generate_model()):
    geo = mod.get_model_geometry()
    geo_numpy = np.array(geo)
    print("\nSize of the matrix containing all the model vertices:")
    print(geo_numpy.shape)
    rep_float = mod.get_model_float_report()
    rep_string = mod.get_model_string_report()
    rep_bool = mod.get_model_bool_report()
    print("\nReport of the generated model:")
    if(len(rep_float)):
        print(rep_float)
    if(len(rep_string)):
        print(rep_string)
    if(len(rep_bool)):
        print(rep_bool)
else:
    print("\nError while instanciating the model generator.")


Size of the matrix containing all the model vertices:
(8, 3)

Report of the generated model:
{'Building Height.0': 11.216289520263672, 'Id': 0.0, 'Max Height.0': 30.0, 'Min Height.0': 10.0, 'Parcel Area.0': 4848.81103515625}


In [7]:
geo_numpy

array([[-26.23299255,   0.        ,  14.82300596],
       [-26.23299255,   0.        ,  56.33999915],
       [ 90.55800018,   0.        ,  56.33999533],
       [ 90.55800018,   0.        ,  14.82299943],
       [-26.23299255,  11.21628952,  14.82300596],
       [-26.23299255,  11.21628952,  56.33999915],
       [ 90.55800018,  11.21628952,  56.33999533],
       [ 90.55800018,  11.21628952,  14.82299943]])

In [5]:
geo_numpy_test = [(-26.23299255,   0.        ,  14.82300596),
       (-26.23299255,   0.        ,  56.33999915),
       ( 90.55800018,   0.        ,  56.33999533),
       ( 90.55800018,   0.        ,  14.82299943),
       (-26.23299255,  11.21628952,  14.82300596),
       (-26.23299255,  11.21628952,  56.33999915),
       ( 90.55800018,  11.21628952,  56.33999533),
       ( 90.55800018,  11.21628952,  14.82299943)]

In [6]:
geo_array = np.array(geo_numpy_test, dtype=np.dtype([('x_coord', float), ('y_coord', float), ('z_coord',float)]))

In [7]:
geo_array

array([(-26.23299255,  0.        , 14.82300596),
       (-26.23299255,  0.        , 56.33999915),
       ( 90.55800018,  0.        , 56.33999533),
       ( 90.55800018,  0.        , 14.82299943),
       (-26.23299255, 11.21628952, 14.82300596),
       (-26.23299255, 11.21628952, 56.33999915),
       ( 90.55800018, 11.21628952, 56.33999533),
       ( 90.55800018, 11.21628952, 14.82299943)],
      dtype=[('x_coord', '<f8'), ('y_coord', '<f8'), ('z_coord', '<f8')])

In [16]:
fc = "output_FeatureClass5"

In [17]:
arcpy.da.NumPyArrayToFeatureClass(geo_array, fc, ("x_coord", "y_coord"))

In [7]:
infc = "export_features_tool.shp"

In [9]:
for row in arcpy.da.SearchCursor(infc,["OID@","SHAPE@"]):
    print("Feature {0}:".format(row[0]))
    partnum = 0

    # Step through each part of the feature
    #
    for part in row[1]:
        # Print the part number
        #
        print("Part {0}:".format(partnum))

        # Step through each vertex in the feature
        #
        for pnt in part:
            if pnt:
                # Print x,y coordinates of current point
                #
                print("{0}, {1}".format(pnt.X, pnt.Y))
            else:
                # If pnt is None, this represents an interior ring
                #
                print("Interior Ring:")
        partnum += 1

Feature 0:
Part 0:
781626.2994192243, 2940401.2469415665
781884.2141394764, 2940185.9036133736
782585.8992281258, 2939606.4267168343
782568.503740102, 2940639.1181097925
782965.6527964771, 2940484.647165656
782932.9196124524, 2938696.198043987
782933.3927004486, 2938631.1260439306
781298.0517709255, 2938134.351131469
781179.0650188178, 2938038.4822033793
780669.0257223397, 2938628.502043918
780604.484666273, 2938701.894043997
780600.3840582818, 2938706.3792919964
780581.3906502575, 2938720.3723800033
780541.6636742204, 2938750.577948034
780500.1778501868, 2938786.995996073
780460.0197701454, 2938826.7042841017
780432.9630021155, 2938850.910364136
780412.4407621026, 2938864.971420139
780382.7530820668, 2938885.0013721585
780361.1416900456, 2938900.7691801786
780328.2678340226, 2938923.5213081986
780266.5486659706, 2938972.1991962492
780237.6940099299, 2938997.019036263
780199.4459459037, 2939023.915164292
780112.7528258264, 2939094.384156361
780086.8769858032, 2939115.2829723805
780066.

In [11]:
spatial_ref = arcpy.Describe(infc).spatialReference

In [14]:
spatial_ref.name

'NAD_1983_StatePlane_Massachusetts_Mainland_FIPS_2001_Feet'

In [15]:
arrayArcPy = arcpy.Array()

In [16]:
for vertex in salut:
    x,y,z = vertex
    pnt = arcpy.Point(x,y,z)
    arrayArcPy.add(pnt)

arrayArcPy.add(arrayArcPy.getObject(0))

In [17]:
arrayArcPy

<Array [<Point (-26.23299255, 0.0, 14.82300596, #)>, <Point (-26.23299255, 0.0, 56.33999915, #)>, <Point (90.55800018, 0.0, 56.33999533, #)>, <Point (90.55800018, 0.0, 14.82299943, #)>, <Point (-26.23299255, 11.21628952, 14.82300596, #)>, <Point (-26.23299255, 11.21628952, 56.33999915, #)>, <Point (90.55800018, 11.21628952, 56.33999533, #)>, <Point (90.55800018, 11.21628952, 14.82299943, #)>, <Point (-26.23299255, 0.0, 14.82300596, #)>]>

In [18]:
boundaryPolygon = arcpy.Polygon(arrayArcPy)

In [24]:
shapefile = "ce_export.shp"
shapefile_clipped = "ce_export_clipped.shp"

In [ ]:
arcpy.Clip_analysis(shapefile, boundaryPolygon, shapefile_clipped)

In [ ]:
print("\nShutdown PRT.")
pyprt.shutdown_prt()